<a href="https://colab.research.google.com/github/abhirammohanmadhav/Image-Caption-Verifier/blob/main/ImageCaptionVerifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install transformers
!pip install diffusers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 81.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 105.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 737.4/737.4 KB 29.0 MB/s eta 0:00:00


In [6]:
from transformers import VisionEncoderDecoderModel, ViTImageProcessor, AutoTokenizer
import torch
from PIL import Image
from diffusers import StableDiffusionPipeline
import os
from skimage.metrics import structural_similarity, mean_squared_error
import cv2

def predict_caption(image_paths, model_type):
    """
    This function is used to generate the caption from images. Here a gpt2-image-captioning
    model is used, taken from huggingface.
    Args:
            image_paths: The path to the input image.
            model_type: The type of model to be used for caption generation
    """
    model = VisionEncoderDecoderModel.from_pretrained(model_type)
    feature_extractor = ViTImageProcessor.from_pretrained(model_type)
    tokenizer = AutoTokenizer.from_pretrained(model_type)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    max_length = 16
    num_beams = 4
    gen_kwargs = {"max_length": max_length, "num_beams": num_beams}

    images = []
    """
    Here PIL.Image library is used instead ofOpenCV. If OpenCV is used, need to convert 
    from BGR to RGB.
    """
    i_image = Image.open(image_paths) 
    if i_image.mode != "RGB":
        i_image = i_image.convert(mode="RGB")

    images.append(i_image)
    print("input image size: ", images[0].size)

    pixel_values = feature_extractor(images=images, return_tensors="pt").pixel_values
    pixel_values = pixel_values.to(device)

    output_ids = model.generate(pixel_values, **gen_kwargs)

    preds = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
    preds = [pred.strip() for pred in preds]
    return images, preds

def generate_image(images, preds, model, output_path, out_img_name):  
    """
    This function is used to generated the image from the caption already generated by 
    predict_caption() function. Here a stable diffusion model v1.5 is used.
    Args:
            images: The list of images from predict_caption() function
            preds: The captions generated for the images.
            model: The type of model to be used for text-to-image generation
            output_path: Root path to store the generated images
            out_img_name: Name of the image to be stored in
    """
    pipe = StableDiffusionPipeline.from_pretrained(model, torch_dtype=torch.float16)
    pipe = pipe.to("cuda")
    image = pipe(preds).images[0]  # captions used from the previous function

    final_path = os.path.join(output_path, out_img_name)
    new_size = images[0].size
    image.resize(new_size)
    print("output image size: ", image.size)
    image.save(final_path)

def similarity_value(ref_image, gen_image, width, height):
    """
    This function is to compare how similar the reference and the generated images are.
    Args:
            ref_image: The path to the reference image.
            gen_image: The path to the generated image.
            width, height: The width and height of the image to be resized to. Both the images 
                                    should be of same dimensions.
    """
    ref_img = cv2.imread(ref_image)
    gen_img = cv2.imread(gen_image)    

    resized_inp_img = cv2.resize(ref_img, (width, height))
    resized_gen_img = cv2.resize(gen_img, (width, height))

    # For SSIM, the images shouls be in gray scale
    ref_img_gray = cv2.cvtColor(resized_inp_img, cv2.COLOR_BGR2GRAY)
    gen_img_gray = cv2.cvtColor(resized_gen_img, cv2.COLOR_BGR2GRAY)

    #SSIM score
    ssim_score, ssim_image = structural_similarity(ref_img_gray, gen_img_gray, full=True)

    #MSE 
    mse = mean_squared_error(resized_inp_img, resized_gen_img)
  
    print('SSIM score:', ssim_score)
    print('MSE score:', mse)
    return ssim_score, mse

if __name__ == "__main__":
    """
    Can generated a config.py python script for larger number of inputs and more complex 
    functions.
    """

    model_type = 'nlpconnect/vit-gpt2-image-captioning' # caption generator
    model_id = 'runwayml/stable-diffusion-v1-5'# image generator
    root_path = f'/content/input_images' # root path for the input image
    image_name = f'image_2.jpg'
    output_path = f'/content'# root path for the output image
    width = 512
    height = 512

    # Here same image name is used for both reference and output images
    final_path = os.path.join(output_path, image_name)
    complete_path = os.path.join(root_path, image_name)

    image, caption = predict_caption(complete_path, model_type)
    generate_image(image, caption, model_id, output_path, image_name)
    ssim, mse = similarity_value(complete_path, final_path)


input image size:  (612, 402)


Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
/usr/local/lib/python3.9/dist-packages/transformers/models/clip/feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(


  0%|          | 0/50 [00:00<?, ?it/s]

output image size:  (512, 512)
SSIM score: 0.19060797904890878
MSE score: 9005.719446818033
